**import tools_conversion1**

In [2]:
from google.colab import files 
src = list(files.upload().values())[0] 


Saving tools_conversion1.py to tools_conversion1.py


In [3]:
import librosa
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tools_conversion1 import scale_in,inv_scaled_out
from tools_conversion1 import audio_to_numpy, numpy_to_matSpectrogram,matSpectrogram_numpy
import soundfile as sf

"""
           from resulted weights and audio noised file (input) , 
           it predicts a denoised voice """
def predicted_voice(weights_path, in_directory, out_directory,
                    in_audio,predicted_audio, 
                    sample_rate, min_duration, 
                    n_fft, hop_length_fft,frame_length,hop_length_frame):

    # load json file and create model
    json_file = open(weights_path+'/'+"model_unet.json", 'r')
    loaded_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_json)
    # load weights into new model
    loaded_model.load_weights(weights_path+'/'+"model_unet.h5")

    # convert noise,voice to numpy
    audio =audio_to_numpy(in_directory,  in_audio, sample_rate,
                          frame_length, hop_length_frame, min_duration)

    #Dimensions of squared spectrograms
    dim_square_spec = int(n_fft / 2) + 1
    print(dim_square_spec)

    # Create magnitude and phase of the sounds
    mat_amp_audio,  mat_pha_audio = numpy_to_matSpectrogram(
        audio, dim_square_spec, n_fft, hop_length_fft)

    #scaling between -1/1
    X_in = scale_in(mat_amp_audio)
    #Reshape for prediction
    X_in = X_in.reshape(X_in.shape[0],X_in.shape[1],X_in.shape[2],1)
    #Prediction
    X_pred = loaded_model.predict(X_in)
    #Rescale back the noise model
    inv_sca_X_pred = inv_scaled_out(X_pred)
    #Remove noise model from noisy speech
    X_denoise = mat_amp_audio - inv_sca_X_pred[:,:,:,0]
    #Reconstruct audio from denoised spectrogram and phase
    print(X_denoise.shape)
    
    audio_denoise_recons = matSpectrogram_numpy(X_denoise, mat_pha_audio, frame_length, hop_length_fft)
    #Number of frames
    nb_samples = audio_denoise_recons.shape[0]
    #Save frames in the same file
    denoise_long = audio_denoise_recons.reshape(1, nb_samples * frame_length)*25
    sf.write(out_directory +predicted_audio , denoise_long[0, :], sample_rate)


**First noised command (moving forward)**
__The name of the audio file: f2.wav

In [ ]:
import IPython 
IPython.display.Audio("/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/test/sound/f2.wav")

**Second noised command (moving backward)**
__The name of the audio file: b.wav

In [ ]:
import IPython
IPython.display.Audio("/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/test/sound/b.wav")

**Denoising process**

In [9]:
##prediction_mode
import IPython
import time
        #path to find pre-trained weights / save models
weights_path = "/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/weights"
        #directory where read noisy sound to denoise
in_directory = "/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/test/sound"
        #directory to save the denoise sound
out_directory ="/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/test/prediction/"

        #noisy sound file
in_audio = ['f2.wav']
        #playing the noised command

        #denoised sound file to save
predicted_audio = "predicted_f2.wav"

        # Minimum duration of audio files
min_duration = 1
        #Frame length 
frame_length = 8064
        # hop length for sound file
hop_length_frame = 8064
        #nb of points for fft(for spectrogram)
n_fft = 255
        #hop length for fft
hop_length_fft = 63
        # Sample rate to read audio
sample_rate=10500
predicted_voice(weights_path, in_directory, out_directory,
                in_audio,predicted_audio, sample_rate, min_duration,n_fft, hop_length_fft,
                frame_length,hop_length_frame,)

IPython.display.Audio("/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/test/prediction/"+predicted_audio,autoplay=True)


128
(5, 128, 128)


**Commands received by the assistant after the denoising process**

In [ ]:
import IPython
IPython.display.Audio ("/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/test/prediction/predicted_f2.wav",autoplay=True)


In [ ]:
import IPython
IPython.display.Audio("/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/test/prediction/predicted_b.wav",autoplay=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
